In [1]:
import pandas as pd

dataset = pd.read_csv("./dataset_100_IBAN_prova.csv",index_col=0)
dataset = dataset.drop("Address",axis=1)
print(dataset.columns)
print(len(dataset))

Index(['BIC', 'AccountNumber', 'CTRYbnk', 'Name', 'IsShared', 'Holder'], dtype='object')
657


In [2]:
import editdistance
from itertools import combinations
from nltk.tokenize import RegexpTokenizer
import pandas as pd
import nltk

#nltk.download('punkt')
tokenizer = RegexpTokenizer(r'\w+')

def remove_punctuation(string):
    """ """
    return " ".join(tokenizer.tokenize(string))

def preprocess_text(string):
    """ """
    removed = remove_punctuation(string)
    return removed.upper().strip()


for i in range(len(dataset)):    
    if isinstance(dataset.loc[i, "Name"],str):
        dataset.loc[i, "Name"] = preprocess_text(dataset.loc[i, "Name"])
    
dataset = dataset.drop_duplicates()

In [5]:
len(dataset)
print(dataset.iloc[0])

BIC                           HMHGETBZ
AccountNumber       NL71BNUZ6616215301
CTRYbnk                             ET
Name                WULFF GMBH CO KGAA
IsShared                             1
Holder           Wulff GmbH & Co. KGaA
Name: 0, dtype: object


In [143]:
import jaro
import numpy as np
from Levenshtein import ratio
from scipy.spatial import distance
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import pairwise_distances


def computeAvgDistanceInNames(group, iban, d):
    """ """
    couples = list(combinations(group["Name"], 2))
    distances = []
    
    if d == "edit":
        for couple in couples:distances.append(ratio(couple[0], couple[1]))
    elif d == "jaro":
        for couple in couples:distances.append(jaro.jaro_winkler_metric(couple[0], couple[1]))
    elif d == "hamming":
        for couple in couples:distances.append(distance.hamming(list(couple[0]), list(couple[1])))
    else:
        raise Exception("Distance not supported")
    
    return sum(distances) / len(distances)


def computeMedoidsAsName(result_set, iban, names, d, clusters=None):
    """ """
    if len(names) == 1:
        result_set[iban]['Names'].append(names[0])
        return result_set
    
    distances = []
    best = np.inf
    best_name = ""
    for name1 in names:
        for name2 in [el for el in names if el != name1]:
            if d == "edit":
                distances.append(ratio(name1, name2))
            elif d == "jaro":
                distances.append(jaro.jaro_winkler_metric(name1, name2))
            elif d == "hamming":
                distances.append(distance.hamming(list(name1), list(name2)))
            else:
                raise Exception("Distance not supported")
        
        actual = sum(distances) / (len(names) - 1)
        if best > actual:
            best = actual
            best_name = name1
    
    #print("PRIMA")
    #print(result_set)
    result_set[iban]['Names'].append(best_name)
    #print("DOPO")
    #print(result_set)
    return result_set



def AgglomerativeC(iban, names, d, cluster_threshold):
    """ """

    # Compute the pairwise distance matrix
    n = len(names)
    distance_matrix = np.zeros((n, n))

    if d == "edit":
        for i in range(n):
            for j in range(i + 1, n):
                dist = ratio(names[i], names[j])
                distance_matrix[i, j] = 1 -dist
                distance_matrix[j, i] = 1 -dist
                #print(names[i], names[j], 1-dist)
                

    # Perform agglomerative clustering
    clustering = AgglomerativeClustering( n_clusters=None, metric='precomputed', linkage='average', distance_threshold=cluster_threshold)
    clusters = clustering.fit_predict(distance_matrix)
    # Print the resulting clusters
    #for idx, cluster in enumerate(clusters):
    #    print(f"String: {names[idx]}, Cluster: {cluster}")    
    
    return clusters

# nomi = ['Seip Rohleder e.G.', 'Karz Henschel AG', 'Gunpf', 'Mälzer KG']
# clusters = AgglomerativeC("", nomi , "edit", 0.4)

# res = list(enumerate(clusters))
# res = [(nomi[el[0]], el[1]) for el in res]
# print(res)
# clusters = list(set(clusters))

# for c in clusters:
#     n = [el[0] for el in res if el[1] == c]
#     print(c, n)
    

[('Seip Rohleder e.G.', 3), ('Karz Henschel AG', 2), ('Gunpf', 1), ('Mälzer KG', 0)]
0 ['Mälzer KG']
1 ['Gunpf']
2 ['Karz Henschel AG']
3 ['Seip Rohleder e.G.']


In [159]:
result_set = {
}

d = "edit"
threshold = 0.1
cluster_threshold = 0.7

groupped = dataset.groupby("AccountNumber")
for iban, group in groupped:
    
    if len(group) == 1:
        result_set[iban] = {
            "Real_IsShared": group.iloc[0]["IsShared"],
            "Holders": list(set(group["Holder"])),
            "IsShared": 0,
            "Names": group["Name"].tolist()
        }
        
    else:
        avg_distance_acc = computeAvgDistanceInNames(group, iban, d)
        #print(avg_distance_acc)
        
        if avg_distance_acc <= threshold:
            result_set[iban] = {
                "Real_IsShared": group.iloc[0]["IsShared"],
                "Holders": list(set(group["Holder"])),
                "IsShared": 0,
                "Names": []
            }
            result_set = computeMedoidsAsName(result_set, iban, group['Name'].tolist(), d)
        else:
            result_set[iban] = {
                "Real_IsShared": group.iloc[0]["IsShared"],
                "Holders": list(set(group["Holder"])),
                "IsShared": 1,
                "Names": []
            }
            name_list = group["Name"].tolist()
            clusters = AgglomerativeC(iban, name_list, d, cluster_threshold)
            res = list(enumerate(clusters))
            res = [(name_list[el[0]], el[1]) for el in res]
            clusters = list(set(clusters))
            
            
            #print(clusters)
            for c in clusters:
                nomi = [el[0] for el in res if el[1] == c]
                #print(nomi)
                result_set = computeMedoidsAsName(result_set, iban, nomi, d)
                
            
accuracy = 0 
for el in result_set:
    if result_set[el]["Real_IsShared"] == result_set[el]["IsShared"]:
        accuracy += 1

print(accuracy / len(result_set) * 100)

60.0
